In [39]:
import cv2
from ultralytics import YOLO
import easyocr
import os
import numpy as np

In [40]:
image_path = '/mnt/c/Users/Rahul/Desktop/Docs and Essentials/Confidential License/mylicense.jpg'
model_path = "/mnt/c/Users/Rahul/Desktop/trained models/License_model.pt"

In [41]:
model = YOLO(model_path)
results = model(image_path)
image = cv2.imread(image_path)
ocr = easyocr.Reader(['ne', 'en'])


image 1/1 /mnt/c/Users/Rahul/Desktop/Docs and Essentials/Confidential License/mylicense.jpg: 352x608 4 citizenship_numbers, 2 contact_numbers, 3 dobs, 2 license_numbers, 1 name, 60.7ms
Speed: 2.5ms preprocess, 60.7ms inference, 2.2ms postprocess per image at shape (1, 3, 352, 608)


In [42]:
class_map = {
    0: 'citizenship_number',
    1: 'contact_number',
    2: 'dob',
    3: 'license_number',
    4: 'name'
}


In [43]:
def get_color(class_id):
    np.random.seed(class_id)
    return tuple(np.random.randint(0, 255, 3).tolist())

In [44]:
base_name = os.path.splitext(os.path.basename(image_path))[0]
output_dir = 'OCR_Outputs'
os.makedirs(output_dir, exist_ok=True)
output_file_path = os.path.join(output_dir, f'{base_name}_results.txt')

collected_texts = {label: [] for label in class_map.values()}

In [45]:
highest_conf_boxes = {}

for result in results:
    boxes = result.boxes.xyxy
    class_ids = result.boxes.cls
    confidences = result.boxes.conf

    for box, class_id, confidence in zip(boxes, class_ids, confidences):
        x1, y1, x2, y2 = map(int, box.tolist())
        label = class_map.get(int(class_id), 'Unknown')
        
        # Update only if this class has not been added yet or a higher confidence is found
        if label not in highest_conf_boxes or confidence > highest_conf_boxes[label][1]:
            highest_conf_boxes[label] = [(x1, y1, x2, y2), confidence]

# Process the highest confidence bounding boxes
for label, (box, confidence) in highest_conf_boxes.items():
    x1, y1, x2, y2 = box
    cropped_img = image[y1:y2, x1:x2]

    ocr_result = ocr.readtext(cropped_img)
    for detection in ocr_result:
        text = detection[1]
        collected_texts[label].append(text)
    
    # Define color and thickness for bounding box and label
    color = get_color(int(class_id))
    thickness = 4  # Thicker bounding box
    font_scale = 1.2  # Larger font size
    font_thickness = 2  # Thicker font

    # Draw bounding box with increased thickness and color
    cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
    cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, font_thickness)


In [46]:

with open(output_file_path, 'w') as file:
    for label, texts in collected_texts.items():
        combined_text = ' '.join(texts)
        if combined_text:
            file.write(f"Class: {label}, Text: {combined_text}\n")

annotated_image_path = os.path.join(output_dir, f'{base_name}_annotated.jpg')
cv2.imwrite(annotated_image_path, image)

True